## 네이버 영화별 영화댓글 크롤러

In [ ]:
import urllib 
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import re
import os
import time
import requests
import pandas as pd
from multiprocessing import Pool

사용된 라이브러리

In [11]:
def get(n):
    base_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?'
    data = []

    def hangle(s):
        return re.sub('[^ ㄱ-ㅣ가-힣]+', '', str(s)).strip()

    def num(s):
        return re.sub('[^ 0-9]+', '', str(s)).strip()

    for i in range(10001, n):
        for page in range(1, 100):
            params = {'code': i,
                      'type': 'after',
                      'isActualPointWriteExecute': 'false',
                      'isMileageSubscriptionAlready': 'false',
                      'isMileageSubscriptionReject': 'false',
                      'page': page}
            url = base_url + urllib.parse.urlencode(params)
            title_url = 'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword={}&target=after&page={}'
            res = requests.get(url)
            title_res = requests.get(title_url.format(i, page))
            clean_text = BeautifulSoup(re.sub("&#(?![0-9])", "", res.text), 'html5lib')
            title_text = BeautifulSoup(re.sub("&#(?![0-9])", "", title_res.text), 'html5lib')
            lis = clean_text.find('div', {'class', 'score_result'}).find('ul')
            score = clean_text.select('div.star_score > em')
            review = clean_text.select('div.score_reple > p')
            title = title_text.select('table.list_netizen > tbody > tr > td.title > a.movie')
            for a, b, c in zip(score, review, title):
                re_score = num(a)
                re_review = hangle(b)
                re_title = hangle(c)
                data.append([re_score, re_review, re_title])
    df = pd.DataFrame(data, columns=['score', 'review', 'title'])
    return df


if __name__ == '__main__':
    pool = Pool(processes=10)
    result = pool.map(get, [10002])
result

[    score                                             review   title
 0      10  배경음  의 감동적인 선율과 영사기에서 나오는 달콤한 키스 장면들을 보며 토토의 회...  시네마 천국
 1       9                   불멸의 명작 영화인을 꿈꾸는 사람이라면 반드시 봐야할 영화  시네마 천국
 2      10  탄탄한 스토리와 거장 엔니오 모리꼬네가 만들어낸 최고의 영화 몇번을 봐도 마지막장면...  시네마 천국
 3      10                       년이 훨씬 지났지만 아직도 생각하면 가슴이 벅차네요  시네마 천국
 4      10                 오랜만에 다시 보아도 너무 예쁘고 사랑스런 눈물나게 하는 영화  시네마 천국
 5      10  지금 극장에 걸려 있는 건 축약판분 축약판만 보신 분들은 꼭 감독판분도 구해서 보시...  시네마 천국
 6       1  영화관에서 절대 보지말아라 이야기를 끌어가는 주요 이야기인 엘레나와 토토의 재회부분...  시네마 천국
 7      10  대에 보았다 그때는 왜 눈물이 그치지 않는지 알수가 없었다 중반 이제서 다시 보고 ...  시네마 천국
 8      10   아 근데 재개봉 버전 뭐이렇게 삭제된 장면이 많냐 후반 엘레나 시퀀스는 통째로 삭제됐네  시네마 천국
 9      10                                  삶의 모든 것이 들어있었다 ㅠㅠ  시네마 천국
 10      9       살바토레와 알프레도의 케미는 가히 역대급이다 명작의 가치는 역시 시대를 초월한다  시네마 천국
 11     10  무슨 네오리얼리즘 영화도 아니고 구지 해석하고 분석하려 하지 말아라그냥 그대로 보고...  시네마 천국
 12      6  신촌에서 예전시네마천국  기대했는데   너무영화가  뚝뚝 끊어지냉 ㄴ누가  편집했는...  시네마 천국
 13     10          

병렬구조로 네이버 영화 댓글 크롤링하는 코드

In [22]:
if len(title):
    ret = ''
    for i in title[0]:
        if i is str:
            ret += i
        else:
            ret += str(i)
    print(ret)

유열의 음악앨범 : 네이버 영화
